In [85]:
import numpy as np
import pandas as pd
import tensorflow as tf
# to make this notebook's output stable across runs
np.random.seed(123)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12


### TensorFlow

Q1. When is a variable initialized? When is it destroyed?

At the beginning of a seession. End of session

Q2. What is the difference between a placeholder and a variable?

You use a variable when you need to store the state of a graph and a placeholder when you want to input some external data

Q3. How many times does reverse-mode autodiff need to traverse the graph in order to compute the gradients of the cost function with regards to 10 variables? What about forward-mode autodiff? And symbolic differentiation?

Define input dimention as n_input, output dimetion as n_output

reverse-mode : n_output + 1

forward-mode : n_input

symbolic differentiation: None

Q4. Implement Logistic Regression with Mini-batch Gradient Descent using TensorFlow. Train it and evaluate it on the moons dataset (introduced in Chapter 5). Try adding all the bells and whistles:

- Define the graph within a logistic_regression() function that can be reused easily.

- Save checkpoints using a Saver at regular intervals during training, and save the final model at the end of training.

- Restore the last checkpoint upon startup if training was interrupted.

- Define the graph using name scopes so the graph looks good in TensorBoard.

- Add summaries to visualize the learning curves in TensorBoard.

- Try tweaking some hyperparameters such as the learning rate or the mini-batch size and look at the shape of the learning curve.

In [3]:
from sklearn.datasets import make_moons

m = 1000
X_moons, y_moons = make_moons(m, noise=0.1, random_state=42)

X_moons_with_bias = np.c_[np.ones((m, 1)), X_moons]
y_moons_column_vector = y_moons.reshape(-1, 1)
test_ratio = 0.2
test_size = int(m * test_ratio)
X_train = X_moons_with_bias[:-test_size]
X_test = X_moons_with_bias[-test_size:]
y_train = y_moons_column_vector[:-test_size]
y_test = y_moons_column_vector[-test_size:]

def random_batch(X_train, y_train, batch_size):
    rnd_indices = np.random.randint(0, len(X_train), batch_size)
    X_batch = X_train[rnd_indices]
    y_batch = y_train[rnd_indices]
    return X_batch, y_batch


In [4]:
X_train_enhanced = np.c_[X_train,
                         np.square(X_train[:, 1]),
                         np.square(X_train[:, 2]),
                         X_train[:, 1] ** 3,
                         X_train[:, 2] ** 3]
X_test_enhanced = np.c_[X_test,
                        np.square(X_test[:, 1]),
                        np.square(X_test[:, 2]),
                        X_test[:, 1] ** 3,
                        X_test[:, 2] ** 3]

In [5]:
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)
reset_graph()


In [6]:
def logistic_regression(X, y, initializer=None, seed=42, learning_rate=0.01):
    n_inputs_including_bias = int(X.get_shape()[1])
    with tf.name_scope("logistic_regression"):
        with tf.name_scope("model"):
            if initializer is None:
                initializer = tf.random_uniform([n_inputs_including_bias, 1], -1.0, 1.0, seed=seed)
            theta = tf.Variable(initializer, name="theta")
            logits = tf.matmul(X, theta, name="logits")
            y_proba = tf.sigmoid(logits)
        with tf.name_scope("train"):
            loss = tf.losses.log_loss(y, y_proba, scope="loss" )
            optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
            training_op = optimizer.minimize(loss)
            loss_summary =tf.summary.scalar('log_loss', loss)
        with tf.name_scope("init"):
            init = tf.global_variables_initializer()
        with tf.name_scope("save"):
            saver = tf.train.Saver()
    return y_proba, loss, training_op, loss_summary, init, saver

In [7]:
from datetime import datetime
import os

def log_dir(prefix=""):
    now = datetime.now().strftime("%Y%m%d%H%M%S")
    root_logdir = "tf_logs"
    if prefix:
        prefix +="-"
    name = prefix + "run-" + now
    return "{}/{}/".format(root_logdir, name)
    

In [8]:
n_inputs = 6
logdir = log_dir("logreg")

X = tf.placeholder(tf.float32, shape=(None, n_inputs+1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")

y_proba, loss, training_op, loss_summary, init, saver = logistic_regression(X, y)

file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [9]:
n_epochs = 10001
batch_size = 50
n_batches = int(np.ceil(m/batch_size))

checkpoint_path = "/tmp/logreg_model.ckpt"
checkpoint_epoch_path = checkpoint_path+".epoch"
final_model_path = "./logreg_model"

with tf.Session() as sess:
    if os.path.isfile(checkpoint_epoch_path):     
        with open(checkpoint_epoch_path, "rb") as f:
            start_epoch = int(f.read())
        print(" Training was interrupted. Check if the checkpoint file exists, restore the model and load the epoch number. Continue the epoch", start_epoch)
        saver.restore(sess, checkpoint_path)
    else:
        start_epoch = 0
        sess.run(init)
    
    for epoch in range(start_epoch, n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = random_batch(X_train_enhanced, y_train, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y:y_batch})
        loss_val, summary_str = sess.run([loss, loss_summary], feed_dict={X: X_test_enhanced, y:y_test})
        file_writer.add_summary(summary_str, epoch)
        
        if epoch %500 ==0:
            print("Epoch:", epoch, "\tLoss:", loss_val)
            saver.save(sess, checkpoint_path)
            with open(checkpoint_epoch_path, 'wb') as f:
                f.write(b"%d" % (epoch+1))
    
    saver.save(sess, final_model_path)
    y_proba_val = y_proba.eval(feed_dict={X: X_test_enhanced, y: y_test})
    os.remove(checkpoint_epoch_path)

Epoch: 0 	Loss: 0.629985
Epoch: 500 	Loss: 0.16122366
Epoch: 1000 	Loss: 0.11903212
Epoch: 1500 	Loss: 0.097329214
Epoch: 2000 	Loss: 0.083697945
Epoch: 2500 	Loss: 0.07437584
Epoch: 3000 	Loss: 0.067502156
Epoch: 3500 	Loss: 0.062206898
Epoch: 4000 	Loss: 0.05802679
Epoch: 4500 	Loss: 0.05456298
Epoch: 5000 	Loss: 0.051708285
Epoch: 5500 	Loss: 0.04923774
Epoch: 6000 	Loss: 0.047167283
Epoch: 6500 	Loss: 0.045376644
Epoch: 7000 	Loss: 0.04381875
Epoch: 7500 	Loss: 0.04237422
Epoch: 8000 	Loss: 0.041089162
Epoch: 8500 	Loss: 0.03997092
Epoch: 9000 	Loss: 0.038920246
Epoch: 9500 	Loss: 0.038010746
Epoch: 10000 	Loss: 0.03715569


In [10]:
y_pred = (y_proba_val >=0.5)

In [11]:
from sklearn.metrics import precision_score, recall_score, accuracy_score
print("precision_score", precision_score(y_test, y_pred))
print("recall_score", recall_score(y_test, y_pred))

precision_score 0.9797979797979798
recall_score 0.9797979797979798


In [13]:
from scipy.stats import reciprocal

n_search_iterations =10

for search_iteration in range(n_search_iterations):
    batch_size = np.random.randint(1,100)
    learning_rate = reciprocal(0.0001, 0.1).rvs(random_state=search_iteration)

    n_inputs =6
    logdir = log_dir("logreg")
    
    print("Iteration", search_iteration)
    print("  logdir:", logdir)
    print("  batch size:", batch_size)
    print("  learning_rate:", learning_rate)
    print("  training: ", end="")
    
    reset_graph()
    

    X = tf.placeholder(tf.float32, shape=(None, n_inputs+1), name="X")
    y = tf.placeholder(tf.float32, shape=(None, 1), name="y")

    y_proba, loss, training_op, loss_summary, init, saver = logistic_regression(X, y,learning_rate=learning_rate)

    file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())
    
    n_epochs = 1001
    n_batches = int(np.ceil(m/batch_size))

    final_model_path = "./logreg_model%d" % search_iteration

    with tf.Session() as sess:
        sess.run(init)

        for epoch in range(start_epoch, n_epochs):
            for batch_index in range(n_batches):
                X_batch, y_batch = random_batch(X_train_enhanced, y_train, batch_size)
                sess.run(training_op, feed_dict={X: X_batch, y:y_batch})
            loss_val, summary_str = sess.run([loss, loss_summary], feed_dict={X: X_test_enhanced, y:y_test})
            file_writer.add_summary(summary_str, epoch)
            if epoch % 500 ==0:
                print(".",end="")                

        saver.save(sess, final_model_path)
        y_proba_val = y_proba.eval(feed_dict={X: X_test_enhanced, y: y_test})
        y_pred =(y_proba_val >= 0.5)
        
        print("precision_score", precision_score(y_test, y_pred))
        print("recall_score", recall_score(y_test, y_pred))

Iteration 0
  logdir: tf_logs/logreg-run-20190423221449/
  batch size: 11
  learning_rate: 0.004430375245218265
  training: ...precision_score 0.97
recall_score 0.9797979797979798
Iteration 1
  logdir: tf_logs/logreg-run-20190423221511/
  batch size: 57
  learning_rate: 0.0017826497151386947
  training: ...precision_score 0.900990099009901
recall_score 0.9191919191919192
Iteration 2
  logdir: tf_logs/logreg-run-20190423221515/
  batch size: 38
  learning_rate: 0.00203228544324115
  training: ...precision_score 0.912621359223301
recall_score 0.9494949494949495
Iteration 3
  logdir: tf_logs/logreg-run-20190423221522/
  batch size: 38
  learning_rate: 0.004491523825137997
  training: ...precision_score 0.95
recall_score 0.9595959595959596
Iteration 4
  logdir: tf_logs/logreg-run-20190423221528/
  batch size: 38
  learning_rate: 0.07963234721775589
  training: ...precision_score 0.9797979797979798
recall_score 0.9797979797979798
Iteration 5
  logdir: tf_logs/logreg-run-20190423221535/
  ba

Q5. Similar to the linear regression implementation in class, write a lasso regression implementation. Use the same dataset, and choose a value for the penalty $\alpha$:

Using a Saver at regular intervals during training, and save the final model at the end of training.

Restore the last checkpoint upon startup if training was interrupted.

Define the graph using name scopes so the graph looks good in TensorBoard.

Add summaries to visualize the learning curves in TensorBoard.

Try tweaking some hyperparameters such as the learning rate or the mini-batch size and look at the shape of the learning curve.

In [64]:
def linear_regression_lasso(X, y, initializer=None, seed=42, learning_rate=0.0001, lasso=0.01):
    n_inputs_including_bias = int(X.get_shape()[1])
    with tf.name_scope("linear_regression_lasso"):
        with tf.name_scope("model"):
            A = tf.Variable(tf.random_normal([7, 1], seed=seed), name="A")
            b = tf.Variable(tf.random_normal([1, 1], seed=seed), name="b")
            y_pred = tf.add(tf.matmul(X, A), b)
        with tf.name_scope("train"):
            lasso_param = tf.constant(lasso)
            loss = tf.add(tf.reduce_mean(tf.square(y - y_pred)), lasso_param * tf.reduce_sum(tf.abs(A)))
            optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
            training_op = optimizer.minimize(loss)
            loss_summary =tf.summary.scalar('lasso_loss', tf.reduce_mean(loss, name = 'loss'))
        with tf.name_scope("init"):
            init = tf.global_variables_initializer()
        with tf.name_scope("save"):
            saver = tf.train.Saver()
    return y_pred, loss, training_op, loss_summary, init, saver

In [65]:
n_inputs = 6
logdir = log_dir("lasso")

X = tf.placeholder(tf.float32, shape=(None, n_inputs+1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")

y_pred, loss, training_op, loss_summary, init, saver = linear_regression_lasso(X, y)

file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [66]:
for batch_index in range(1):
    X_batch_1, y_batch_1 = random_batch(X_train_enhanced, y_train, batch_size)

In [67]:
with tf.Session() as sess:
    sess.run(init)
    #sess.run(training_op, feed_dict={X: X_train_enhanced, y:y_train})


        
    sess.run(training_op, feed_dict={X: X_batch_1, y:y_batch_1})    
    loss_val = sess.run([loss], feed_dict={X: X_test_enhanced, y:y_test})    
    y_pred_val = y_pred.eval(feed_dict={X: X_test_enhanced, y: y_test})
    print(loss_val)
    print(y_pred_val.shape)
    #rp = sess.run(regularization_param)
    #sess.run(training_op, feed_dict={X: X_batch_1, y:y_batch_1})    
    #loss_val = sess.run([loss], feed_dict={X: X_test_enhanced, y:y_test})    
    #y_pred_val = y_pred.eval(feed_dict={X: X_test_enhanced, y: y_test})
    #print(loss_val)
    #print(rp)
        
        
    
    #loss_val, summary_str = sess.run([loss, loss_summary], feed_dict={X: X_test_enhanced, y:y_test})
    

[5.301681]
(200, 1)


In [68]:
n_epochs = 2000
batch_size = 50
n_batches = int(np.ceil(m/batch_size))
print(n_batches)
checkpoint_path = "/tmp/lasso_model.ckpt"
checkpoint_epoch_path = checkpoint_path+".epoch"
final_model_path = "./lasso_model"

with tf.Session() as sess:
    learning_rate = 0.001
    
    y_pred, loss, training_op, loss_summary, init, saver = linear_regression_lasso(X,y, learning_rate=learning_rate)
    #y_pred, loss, training_op, loss_summary, init, saver = logistic_regression(X,y, learning_rate=learning_rate)
    
    file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())
    
    if os.path.isfile(checkpoint_epoch_path):     
        #with open(checkpoint_epoch_path, "rb") as f:
        #    start_epoch = int(f.read())
        #print(" Training was interrupted. Check if the checkpoint file exists, restore the model and load the epoch number. Continue the epoch", start_epoch)
        #saver.restore(sess, checkpoint_path)
        sess.run(init)
    else:
        start_epoch = 0
        sess.run(init)
    print(loss_summary)
    for epoch in range(start_epoch, n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = random_batch(X_train_enhanced, y_train, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y:y_batch})
        loss_val, summary_str = sess.run([loss, loss_summary], feed_dict={X: X_test_enhanced, y:y_test})
        file_writer.add_summary(summary_str, epoch)
        
        if epoch %200 ==0:
            print("Epoch:", epoch, "\tLoss:", loss_val)
            saver.save(sess, checkpoint_path)
            with open(checkpoint_epoch_path, 'wb') as f:
                f.write(b"%d" % (epoch+1))
    
    saver.save(sess, final_model_path)
    y_pred_val = y_pred.eval(feed_dict={X: X_test_enhanced, y: y_test})
    y_cla = (y_pred_val>=0.5)
    os.remove(checkpoint_epoch_path)

20
Tensor("linear_regression_lasso_8/train/lasso_loss:0", shape=(), dtype=string)
Epoch: 0 	Loss: 4.518378
Epoch: 200 	Loss: 0.10023286
Epoch: 400 	Loss: 0.084871136
Epoch: 600 	Loss: 0.07698221
Epoch: 800 	Loss: 0.072679766
Epoch: 1000 	Loss: 0.06944487
Epoch: 1200 	Loss: 0.06733396
Epoch: 1400 	Loss: 0.06626691
Epoch: 1600 	Loss: 0.065731764
Epoch: 1800 	Loss: 0.064936966


In [69]:
print("precision_score", precision_score(y_test, y_cla))
print("recall_score", recall_score(y_test, y_cla))

precision_score 0.9693877551020408
recall_score 0.9595959595959596


In [38]:
from tensorflow_graph_in_jupyter import show_graph

In [39]:
show_graph(tf.get_default_graph())

In [77]:
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [78]:
reset_graph()

from datetime import datetime





In [84]:
from scipy.stats import reciprocal
from sklearn.metrics import precision_score, recall_score

n_search_iterations =10

for search_iteration in range(n_search_iterations):
    reset_graph()
    file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())
    batch_size = np.random.randint(1,100)
    learning_rate = reciprocal(0.0001, 0.1).rvs(random_state=search_iteration)

    n_inputs =6
    now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
    root_logdir = "tf_logs"
    logdir = "{}/run-{}/".format(root_logdir, now)

    print("Iteration", search_iteration)
    print("  logdir:", logdir)
    print("  batch size:", batch_size)
    print("  learning_rate:", learning_rate)
    print("  training: ", end="")
    


    X = tf.placeholder(tf.float32, shape=(None, n_inputs+1), name="X")
    y = tf.placeholder(tf.float32, shape=(None, 1), name="y")

    y_pred, loss, training_op, loss_summary, init, saver = linear_regression_lasso(X, y,learning_rate=learning_rate)
    with tf.name_scope("loss") as scope:
        error = y_pred - y
        mse = tf.reduce_mean(tf.square(error), name="mse")
    file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())
    
    n_epochs = 10001
    n_batches = int(np.ceil(m/batch_size))

    final_model_path = "./logreg_model%d" % search_iteration
    
    with tf.Session() as sess:
        sess.run(init)

        for epoch in range(n_epochs):
            for batch_index in range(n_batches):
                X_batch, y_batch = random_batch(X_train_enhanced, y_train, batch_size)
                sess.run(training_op, feed_dict={X: X_batch, y:y_batch})
            loss_val, summary_str = sess.run([loss, loss_summary], feed_dict={X: X_test_enhanced, y:y_test})
            file_writer.add_summary(summary_str, epoch)
            #print(loss_val)
            if epoch % 500 ==0:
                print(".",end="")                
        saver.save(sess, final_model_path)
        y_pred_val = y_pred.eval(feed_dict={X: X_test_enhanced, y: y_test})
        y_pred_cla =(y_pred_val >= 0.5)
        #print(y_pred_val)
        print("precision_score", precision_score(y_test, y_pred_cla))
        print("recall_score", recall_score(y_test, y_pred_cla))
        
        file_writer.flush()
        file_writer.close()

Iteration 0
  logdir: tf_logs/run-20190423143529/
  batch size: 52
  learning_rate: 0.004430375245218265
  training: ..precision_score 0.9693877551020408
recall_score 0.9595959595959596
Iteration 1
  logdir: tf_logs/run-20190423143531/
  batch size: 52
  learning_rate: 0.0017826497151386947
  training: ..precision_score 0.9238095238095239
recall_score 0.9797979797979798
Iteration 2
  logdir: tf_logs/run-20190423143533/
  batch size: 52
  learning_rate: 0.00203228544324115
  training: ..precision_score 0.9326923076923077
recall_score 0.9797979797979798
Iteration 3
  logdir: tf_logs/run-20190423143536/
  batch size: 52
  learning_rate: 0.004491523825137997
  training: .

KeyboardInterrupt: 